In [1]:
import json
import time
from itertools import islice
from datetime import datetime
from streamReader import streamReader
from htm_anomaly_detection import HTM
from nupic.engine import Network
from nupic.encoders import MultiEncoder, ScalarEncoder, DateEncoder

### Before run the code below, make sure you're running "step1. data simulator.ipynb" to generate the data cache
### Example1. use it without loading saved model

In [2]:
'''
parameters that we need to define
'''
_TIMEOFDAY = (21,4)  # timeofday = internal buffer, see this: https://nupic.docs.numenta.org/1.0.3/api/algorithms/encoders.html
_SAVEMODEL = False
_USE_SAVED_MODEL = False


'''
Initialize HTM anomaly detection object
'''
htm = HTM(use_saved_model = _USE_SAVED_MODEL)


'''
Run the code below once to set encoders, SP, and TM parameters. 
After this, if you don't want to change these parameters then you should just read them from files.
'''
encoder_params = {
   'scalarEncoder1Args':{
     "minval": 0,
     "maxval": 20,
     "w": 21,
     "periodic": False,
     "n": 50,
     "radius": 0,
     "resolution": 0,
     "name": "value",
     "verbosity": 0,
     "clipInput": False,
     "forced": False,
   },
       
   'dateEncoder1Args':{
     "season": 0,
     "dayOfWeek": 0,
     "weekend": 0,
     "holiday": 0,
     "timeOfDay": _TIMEOFDAY,
     "customDays": 0,
     "name": "time",
     "forced": False
   },
       
   'scalarEncoder2Args':{
     "minval": 0,
     "maxval": 20,
     "w": 21,
     "periodic": False,
     "n": 50,
     "radius": 0,
     "resolution": 0,
     "name": "value",
     "verbosity": 0,
     "clipInput": False,
     "forced": False,
   },
   
   'dateEncoder2Args':{
     "season": 0,
     "dayOfWeek": 0,
     "weekend": 0,
     "holiday": 0,
     "timeOfDay": _TIMEOFDAY,
     "customDays": 0,
     "name": "time",
     "forced": False
   }
}
       
htm.setEncoderParams('./encoders.json', encoder_params)

_SP_PARAMS = {
   'SP':{
       "spatialImp": "cpp",
       "globalInhibition": 1,
       "columnCount": 2048,
       "inputWidth": 0,
       "numActiveColumnsPerInhArea": 40,
       "seed": 1956,
       "potentialPct": 0.8,
       "synPermConnected": 0.1,
       "synPermActiveInc": 0.0001,
       "synPermInactiveDec": 0.0005,
       "boostStrength": 0.0,
   }
}

_TM_PARAMS = {
   'TM':{
       "columnCount": 2048,
       "cellsPerColumn": 32,
       "inputWidth": 2048,
       "seed": 1960,
       "temporalImp": "cpp",
       "newSynapseCount": 20,
       "maxSynapsesPerSegment": 32,
       "maxSegmentsPerCell": 128,
       "initialPerm": 0.21,
       "permanenceInc": 0.1,
       "permanenceDec": 0.1,
       "globalDecay": 0.0,
       "maxAge": 0,
       "minThreshold": 9,
       "activationThreshold": 12,
       "outputType": "normal",
       "pamLength": 3,
   }
}

htm.setEncoderParams('./SP.json', _SP_PARAMS)
htm.setEncoderParams('./TM.json', _TM_PARAMS)

In [3]:
'''
Load the above parameters from json files and build data stream reader to read data from cache.
The data has to be in the exact format of cache generated by "Step1. data simulator.ipynb"
'''

scalarEncoder1Args = htm.getEncoderParams('./encoders.json', 'scalarEncoder1Args')
dateEncoder1Args = htm.getEncoderParams('./encoders.json', 'dateEncoder1Args')
scalarEncoder2Args = htm.getEncoderParams('./encoders.json', 'scalarEncoder2Args')
dateEncoder2Args = htm.getEncoderParams('./encoders.json', 'dateEncoder2Args')
SPArgs = htm.getEncoderParams('./SP.json', 'SP')
TMArgs = htm.getEncoderParams('./TM.json', 'TM')

input01_recordParams = {
  "scalarEncoderArgs": scalarEncoder1Args,
  "dateEncoderArgs": dateEncoder1Args,
}

input02_recordParams = {
  "scalarEncoderArgs": scalarEncoder2Args,
  "dateEncoderArgs": dateEncoder2Args,
}

# define the data format in cache
names = ['time', 'value']
types = ['datetime', 'float']
specials = ['T', '']
streamReader1 = streamReader(streamID = './datacache01', names = names, types = types, specials = specials)
streamReader2 = streamReader(streamID = './datacache02', names = names, types = types, specials = specials)

In [4]:
'''
create networks and tell them the source of data for prediction
'''
network01 = htm.createNetwork(datasource=streamReader1, recordParams=input01_recordParams, spatialParams=SPArgs, temporalParams=TMArgs)
network02 = htm.createNetwork(datasource=streamReader2, recordParams=input02_recordParams, spatialParams=SPArgs, temporalParams=TMArgs)

In [5]:
'''
Looping network.run() to get iterative prediction from data cache.
The generator will keep writing to data cache so while(1) loop will not touch to the end of bottom.
I use 10 iteration here to show you how to save the model.
'''
iteration = 0
while(iteration < 10):
    fed_in_data01, anomalyLikelihood1 = htm.run(network01)
    fed_in_data02, anomalyLikelihood2 = htm.run(network02)
    print 'fed_in_data01',fed_in_data01,' anomaly likelihood:',anomalyLikelihood1
    print 'fed_in_data01',fed_in_data02,' anomaly likelihood:',anomalyLikelihood2, '\n'
    iteration += 1
    time.sleep(2)
htm.save_network(network01, './models/network1.nta')
htm.save_network(network02, './models/network2.nta')

fed_in_data01 11.4441  anomaly likelihood: 0.5
fed_in_data01 10.9207  anomaly likelihood: 0.5 

fed_in_data01 13.4668  anomaly likelihood: 0.5
fed_in_data01 2.66524  anomaly likelihood: 0.5 

fed_in_data01 4.23665  anomaly likelihood: 0.5
fed_in_data01 11.1507  anomaly likelihood: 0.5 

fed_in_data01 14.4405  anomaly likelihood: 0.5
fed_in_data01 2.91072  anomaly likelihood: 0.5 

fed_in_data01 2.85113  anomaly likelihood: 0.5
fed_in_data01 6.16676  anomaly likelihood: 0.5 

fed_in_data01 14.8674  anomaly likelihood: 0.5
fed_in_data01 12.1911  anomaly likelihood: 0.5 

fed_in_data01 7.50275  anomaly likelihood: 0.5
fed_in_data01 3.52322  anomaly likelihood: 0.5 

fed_in_data01 11.9118  anomaly likelihood: 0.5
fed_in_data01 9.50803  anomaly likelihood: 0.5 

fed_in_data01 4.49895  anomaly likelihood: 0.5
fed_in_data01 0.191487  anomaly likelihood: 0.5 

fed_in_data01 7.61852  anomaly likelihood: 0.5
fed_in_data01 10.9783  anomaly likelihood: 0.5 



### Example2. use it by loading saved model

In [6]:
'''
Comming soon...
For the current data stream reader, there are still some problems associated with this part.
'''

# _USE_SAVED_MODEL = True
# htm = HTM(use_saved_model = _USE_SAVED_MODEL)

# scalarEncoder1Args = htm.getEncoderParams('./encoders.json', 'scalarEncoder1Args')
# dateEncoder1Args = htm.getEncoderParams('./encoders.json', 'dateEncoder1Args')
# scalarEncoder2Args = htm.getEncoderParams('./encoders.json', 'scalarEncoder2Args')
# dateEncoder2Args = htm.getEncoderParams('./encoders.json', 'dateEncoder2Args')
# SPArgs = htm.getEncoderParams('./SP.json', 'SP')
# TMArgs = htm.getEncoderParams('./TM.json', 'TM')

# input01_recordParams = {
#   "scalarEncoderArgs": scalarEncoder1Args,
#   "dateEncoderArgs": dateEncoder1Args,
# }

# input02_recordParams = {
#   "scalarEncoderArgs": scalarEncoder2Args,
#   "dateEncoderArgs": dateEncoder2Args,
# }

# # names = ['time', 'value']
# # types = ['datetime', 'float']
# # specials = ['T', '']
# # streamReader1 = streamReader(streamID = './datacache01', names = names, types = types, specials = specials)
# # streamReader2 = streamReader(streamID = './datacache02', names = names, types = types, specials = specials)

# model1 = './models/network1.nta'
# model2 = './models/network2.nta'
# network01 = htm.createNetwork(datasource=None, recordParams=input01_recordParams, spatialParams=None, temporalParams=None, model_path = model1)
# network02 = htm.createNetwork(datasource=None, recordParams=input02_recordParams, spatialParams=None, temporalParams=None, model_path = model2)

# iteration = 0
# while(iteration < 5):
#     fed_in_data01, anomalyLikelihood1 = htm.run(network01)
#     fed_in_data02, anomalyLikelihood2 = htm.run(network02)
#     print 'fed_in_data01',fed_in_data01,' anomaly likelihood:',anomalyLikelihood1
#     print 'fed_in_data01',fed_in_data02,' anomaly likelihood:',anomalyLikelihood2, '\n'
#     iteration += 1
#     time.sleep(2)

'\nComming soon...\nFor the current data stream reader, there are still some problems associated with this part.\n'